In [2]:
from numpy.random import multinomial
from numpy import log, exp
from numpy import argmax
import json
import numpy as np


class MovieGroupProcess:
    def __init__(self, K=8, alpha=0.1, beta=0.1, n_iters=30):
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.n_iters = n_iters

        # slots for computed variables
        self.number_docs = None
        self.vocab_size = None
        self.cluster_doc_count = [0 for _ in range(K)]
        self.cluster_word_count = [0 for _ in range(K)]
        self.cluster_word_distribution = [{} for i in range(K)]

    @staticmethod
    def from_data(K, alpha, beta, D, vocab_size, cluster_doc_count, cluster_word_count, cluster_word_distribution):
        '''
        Reconstitute a MovieGroupProcess from previously fit data
        :param K:
        :param alpha:
        :param beta:
        :param D:
        :param vocab_size:
        :param cluster_doc_count:
        :param cluster_word_count:
        :param cluster_word_distribution:
        :return:
        '''
        mgp = MovieGroupProcess(K, alpha, beta, n_iters=30)
        mgp.number_docs = D
        mgp.vocab_size = vocab_size
        mgp.cluster_doc_count = cluster_doc_count
        mgp.cluster_word_count = cluster_word_count
        mgp.cluster_word_distribution = cluster_word_distribution
        return mgp

    @staticmethod
    def _sample(p):
        '''
        Sample with probability vector p from a multinomial distribution
        :param p: list
            List of probabilities representing probability vector for the multinomial distribution
        :return: int
            index of randomly selected output
        '''
        return [i for i, entry in enumerate(multinomial(1, p)) if entry != 0][0]

    def fit(self, docs, vocab_size):
        '''
        Cluster the input documents
        :param docs: list of list
            list of lists containing the unique token set of each document
        :param V: total vocabulary size for each document
        :return: list of length len(doc)
            cluster label for each document
        '''
        alpha, beta, K, n_iters, V = self.alpha, self.beta, self.K, self.n_iters, vocab_size

        D = len(docs)
        self.number_docs = D
        self.vocab_size = vocab_size

        # unpack to easy var names
        m_z, n_z, n_z_w = self.cluster_doc_count, self.cluster_word_count, self.cluster_word_distribution
        cluster_count = K
        d_z = [None for i in range(len(docs))]

        # initialize the clusters
        for i, doc in enumerate(docs):

            # choose a random  initial cluster for the doc
            z = self._sample([1.0 / K for _ in range(K)])
            d_z[i] = z
            m_z[z] += 1
            n_z[z] += len(doc)

            for word in doc:
                if word not in n_z_w[z]:
                    n_z_w[z][word] = 0
                n_z_w[z][word] += 1

        for _iter in range(n_iters):
            total_transfers = 0

            for i, doc in enumerate(docs):

                # remove the doc from it's current cluster
                z_old = d_z[i]

                m_z[z_old] -= 1
                n_z[z_old] -= len(doc)

                for word in doc:
                    n_z_w[z_old][word] -= 1

                    # compact dictionary to save space
                    if n_z_w[z_old][word] == 0:
                        del n_z_w[z_old][word]

                # draw sample from distribution to find new cluster
                p = self.score(doc)
                z_new = self._sample(p)

                # transfer doc to the new cluster
                if z_new != z_old:
                    total_transfers += 1

                d_z[i] = z_new
                m_z[z_new] += 1
                n_z[z_new] += len(doc)

                for word in doc:
                    if word not in n_z_w[z_new]:
                        n_z_w[z_new][word] = 0
                    n_z_w[z_new][word] += 1

            cluster_count_new = sum([1 for v in m_z if v > 0])
            print("In stage %d: transferred %d clusters with %d clusters populated" % (
            _iter, total_transfers, cluster_count_new))
            if total_transfers == 0 and cluster_count_new == cluster_count and _iter>25:
                print("Converged.  Breaking out.")
                break
            cluster_count = cluster_count_new
        self.cluster_word_distribution = n_z_w
        return d_z

    def score(self, doc):
        '''
        Score a document

        Implements formula (3) of Yin and Wang 2014.
        http://dbgroup.cs.tsinghua.edu.cn/wangjy/papers/KDD14-GSDMM.pdf

        :param doc: list[str]: The doc token stream
        :return: list[float]: A length K probability vector where each component represents
                              the probability of the document appearing in a particular cluster
        '''
        alpha, beta, K, V, D = self.alpha, self.beta, self.K, self.vocab_size, self.number_docs
        m_z, n_z, n_z_w = self.cluster_doc_count, self.cluster_word_count, self.cluster_word_distribution

        p = [0 for _ in range(K)]

        #  We break the formula into the following pieces
        #  p = N1*N2/(D1*D2) = exp(lN1 - lD1 + lN2 - lD2)
        #  lN1 = log(m_z[z] + alpha)
        #  lN2 = log(D - 1 + K*alpha)
        #  lN2 = log(product(n_z_w[w] + beta)) = sum(log(n_z_w[w] + beta))
        #  lD2 = log(product(n_z[d] + V*beta + i -1)) = sum(log(n_z[d] + V*beta + i -1))

        lD1 = log(D - 1 + K * alpha)
        doc_size = len(doc)
        for label in range(K):
            lN1 = log(m_z[label] + alpha)
            lN2 = 0
            lD2 = 0
            for word in doc:
                lN2 += log(n_z_w[label].get(word, 0) + beta)
            for j in range(1, doc_size +1):
                lD2 += log(n_z[label] + V * beta + j - 1)
            p[label] = exp(lN1 - lD1 + lN2 - lD2)

        # normalize the probability vector
        pnorm = sum(p)
    
        pnorm = pnorm if pnorm>0 else 1
        return [pp/pnorm for pp in p]

    def choose_best_label(self, doc):
        '''
        Choose the highest probability label for the input document
        :param doc: list[str]: The doc token stream
        :return:
        '''
        p = self.score(doc)
        return argmax(p),max(p)

In [3]:
def compute_V(texts):
    V = set()
    for text in texts:
        for word in text:
            V.add(word)
    return len(V)

In [4]:
text = "본 장에서는 일반 대출 신청자 및 씬파일러 데이터 세트에서 베이스라인 모델과 TeGCN 모델 간 성능 비교를 수행했다. 이 때, 모델의 일반화 가능성을 검증하기 위해 K-Fold 교차검증을 수행하였다. 교차검증은 총 5회 실시하였으며, 각 Fold의 평균을 통해 도출한 실험 결과는 표 5와 같다. TeGCN 모델은 일반 대출 신청자 데이터에 대하여 5개의 머신러닝 및 딥러닝 기반 베이스라인 모델보다 성능이 뛰어난 것으로 나타났다. AUC 값과, K-S 통계량 두 평가 지표에서 모두 TeGCN 모델이 우수한 것으로 나타났으며, 이는 그래프 정보를 활용한 신용 평가 모형이 채무 불이행 예측이 유의미한 것을 나타낸다. 또한, 금융 이력이 부족한 씬파일러 데이터 세트에서 역시 베이스라인 모델 대비 높은 성능을 보였다. 특히, 일부 베이스라인 모델들이 씬파일러 예측 시 성능이 떨어진데 반해, 본 모델은 비교적 안정적인 성능을 유지했다. 전체 데이터 세트와 마찬가지로 씬파일러 데이터에서도 기준 모델보다 TeGCN 모델의 성능이 뛰어난 것으로 나타났다. 마지막으로, TeGCN은 Tab Transformer 대신 MLP 레이어를 추가한 GCN 모델 대비 뛰어난 성능을 기록했다. 이는 Tab Transformer를 활용한 범주형 변수 임베딩이 채무 불이행 예측 성능 향상에 유의미한 도움이 된다는 것을 의미한다. 추가적으로, 타겟 클래스 균형에 대한 모델의 영향과 성능을 확인하기 위하여 소수 클래스와 다수 클래스의 비율을 1:1로 맞춰 모델에 대한 Sub-Sample 테스트를 아래 표 6과 같이 진행하였다. 그 결과, 클래스 균형의 상황에서도 모델은 기존 모델 대비 준수한 성능을 보였다. 본 연구는 모델 학습에 활용된 23개의 핵심 변수들의 중요도를 분석하였다. 중요도 평가를 위하여 Feature Permutation 방식을 적용하였다. 이 방법은 특정 변수의 값을 임의로 섞어, 기존 출력과 변경된 출력 사이의 차이로 해당 변수의 중요도를 측정한다. 만약 해당 변수 값을 변경해도 모델 출력에 크게 영향을 미치지 않는다면, 그 변수는 상대적으로 중요하지 않다는 해석이 가능하다. 그러나 큰 차이가 발생한다면, 해당 변수는 모델에서 핵심적인 역할을 수행한다고 볼 수 있다. Feature Permutation 방식을 통해 확인한 변수 중요도는 그림 3과 같다. ‘분할 납부금(installment)’과 ‘대출 금액(loan_amount)’등의 주요 대출 관련 변수는 채무 불이행 예측에 결정적인 영향을 주는 것으로 나타났다. 그 외에도 ‘리볼빙 서비스 이용률 (revlo util)’과 ‘모기지 계좌 수(mort_acc)’ 그리고 ‘신용계좌 수(open_acc)’와 같은 변수들 또한 중요한 역할을 하는 것으로 평가되었다."

sentences = text.split('. ')
quoted_sentences = ['"' + sentence + '"' for sentence in sentences]

print(quoted_sentences)

['"본 장에서는 일반 대출 신청자 및 씬파일러 데이터 세트에서 베이스라인 모델과 TeGCN 모델 간 성능 비교를 수행했다"', '"이 때, 모델의 일반화 가능성을 검증하기 위해 K-Fold 교차검증을 수행하였다"', '"교차검증은 총 5회 실시하였으며, 각 Fold의 평균을 통해 도출한 실험 결과는 표 5와 같다"', '"TeGCN 모델은 일반 대출 신청자 데이터에 대하여 5개의 머신러닝 및 딥러닝 기반 베이스라인 모델보다 성능이 뛰어난 것으로 나타났다"', '"AUC 값과, K-S 통계량 두 평가 지표에서 모두 TeGCN 모델이 우수한 것으로 나타났으며, 이는 그래프 정보를 활용한 신용 평가 모형이 채무 불이행 예측이 유의미한 것을 나타낸다"', '"또한, 금융 이력이 부족한 씬파일러 데이터 세트에서 역시 베이스라인 모델 대비 높은 성능을 보였다"', '"특히, 일부 베이스라인 모델들이 씬파일러 예측 시 성능이 떨어진데 반해, 본 모델은 비교적 안정적인 성능을 유지했다"', '"전체 데이터 세트와 마찬가지로 씬파일러 데이터에서도 기준 모델보다 TeGCN 모델의 성능이 뛰어난 것으로 나타났다"', '"마지막으로, TeGCN은 Tab Transformer 대신 MLP 레이어를 추가한 GCN 모델 대비 뛰어난 성능을 기록했다"', '"이는 Tab Transformer를 활용한 범주형 변수 임베딩이 채무 불이행 예측 성능 향상에 유의미한 도움이 된다는 것을 의미한다"', '"추가적으로, 타겟 클래스 균형에 대한 모델의 영향과 성능을 확인하기 위하여 소수 클래스와 다수 클래스의 비율을 1:1로 맞춰 모델에 대한 Sub-Sample 테스트를 아래 표 6과 같이 진행하였다"', '"그 결과, 클래스 균형의 상황에서도 모델은 기존 모델 대비 준수한 성능을 보였다"', '"본 연구는 모델 학습에 활용된 23개의 핵심 변수들의 중요도를 분석하였다"', '"중요도 평가를 위하여 Feature Permutation 방식을 적용하였다"', '"이 방법은 특정 변수의 값을 임의로 

In [4]:
from konlpy.tag import Okt, Hannanum, Kkma,Mecab

# 각 형태소 분석기를 초기화
okt = Okt()
hannanum = Hannanum()
kkma = Kkma()
mecab = Mecab()

# 명사 추출 함수 정의
def extract_nouns(sentences):
    result = {
        "mecab": [],
        "okt": [],
        "hannanum": [],
        "kkma": []
    }
    for sentence in sentences:
        result["mecab"].append(mecab.nouns(sentence))
        result["okt"].append(okt.nouns(sentence))
        result["hannanum"].append(hannanum.nouns(sentence))
        result["kkma"].append(kkma.nouns(sentence))
    return result

# 명사 추출 실행
nouns_extracted = extract_nouns(quoted_sentences)

# 결과 출력
for analyzer, nouns_list in nouns_extracted.items():
    print(f"{analyzer} 명사 추출:")
    for i, nouns in enumerate(nouns_list):
        print(f"문장 {i+1}: {nouns}")
    print()

mecab 명사 추출:
문장 1: ['장', '일반', '대출', '신청자', '씬', '파일러', '데이터', '세트', '베이스', '라인', '모델', '모델', '간', '성능', '비교', '수행']
문장 2: ['때', '모델', '일반', '가능', '검증', '교차', '검증', '수행']
문장 3: ['교차', '검증', '회', '실시', '평균', '도출', '실험', '결과', '표']
문장 4: ['모델', '일반', '대출', '신청자', '데이터', '개', '머신', '러닝', '딥', '러닝', '기반', '베이스', '라인', '모델', '성능', '것']
문장 5: ['값', '통계량', '평가', '지표', '모델', '것', '이', '그래프', '정보', '활용', '신용', '평가', '모형', '채무', '불이행', '예측', '것']
문장 6: ['금융', '이력', '부족', '씬', '파일러', '데이터', '세트', '베이스', '라인', '모델', '대비', '성능']
문장 7: ['일부', '베이스', '라인', '모델', '씬', '파일러', '예측', '시', '성능', '데', '모델', '비교', '안정', '성능', '유지']
문장 8: ['전체', '데이터', '세트', '마찬가지', '씬', '파일러', '데이터', '기준', '모델', '모델', '성능', '것']
문장 9: ['마지막', '대신', '레이어', '추가', '모델', '대비', '성능', '기록']
문장 10: ['이', '활용', '범주', '변수', '딩', '채무', '불이행', '예측', '성능', '향상', '도움', '것', '의미']
문장 11: ['추가', '클래스', '균형', '모델', '영향', '성능', '확인', '소수', '클래스', '다수', '클래스', '비율', '모델', '테스트', '아래', '표', '과', '진행']
문장 12: ['결과', '클래스', '균형', '상황', '모델', '기

In [6]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print("\nCluster %s : %s"%(cluster,sort_dicts))

In [30]:
from konlpy.tag import Hannanum

# Hannanum 초기화
hannanum = Hannanum()

# Hannanum 명사 추출 및 텍스트 분리 함수
def extract_hannanum_nouns(sentences):
    return [hannanum.nouns(sentence) for sentence in sentences]

# Hannanum 명사 추출 결과
hannanum_nouns = extract_hannanum_nouns(quoted_sentences)

# 공백으로 분리된 텍스트 리스트 생성
texts = [" ".join(nouns).split() for nouns in hannanum_nouns]

mgp = MovieGroupProcess(K=20, n_iters=200, alpha=0.1, beta=0.1)
V = compute_V(texts)
y = mgp.fit(texts, V)
print(y)

print("----------------------------------- RESULT -----------------------------------")
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster

top_words(mgp.cluster_word_distribution, top_index, 10)

In stage 0: transferred 14 clusters with 9 clusters populated
In stage 1: transferred 7 clusters with 8 clusters populated
In stage 2: transferred 4 clusters with 7 clusters populated
In stage 3: transferred 5 clusters with 10 clusters populated
In stage 4: transferred 6 clusters with 8 clusters populated
In stage 5: transferred 5 clusters with 8 clusters populated
In stage 6: transferred 6 clusters with 9 clusters populated
In stage 7: transferred 7 clusters with 8 clusters populated
In stage 8: transferred 4 clusters with 8 clusters populated
In stage 9: transferred 5 clusters with 9 clusters populated
In stage 10: transferred 4 clusters with 9 clusters populated
In stage 11: transferred 5 clusters with 9 clusters populated
In stage 12: transferred 5 clusters with 9 clusters populated
In stage 13: transferred 6 clusters with 8 clusters populated
In stage 14: transferred 6 clusters with 10 clusters populated
In stage 15: transferred 7 clusters with 9 clusters populated
In stage 16: tr

In [15]:
from konlpy.tag import Mecab
mecab = Mecab()

# Function to extract nouns using Mecab
def extract_mecab_nouns(sentences):
    return [mecab.nouns(sentence) for sentence in sentences]

# Extract nouns using Mecab
mecab_nouns = extract_mecab_nouns(quoted_sentences)
print(mecab_nouns)
texts = mecab_nouns
texts = [" ".join(nouns).split() for nouns in mecab_nouns]



mgp = MovieGroupProcess(K=20, n_iters=200, alpha=0.1, beta=0.1)
V = compute_V(texts)
y = mgp.fit(texts, V)

print(y)

print("----------------------------------- RESULT -----------------------------------")
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 10)

[['장', '일반', '대출', '신청자', '씬', '파일러', '데이터', '세트', '베이스', '라인', '모델', '모델', '간', '성능', '비교', '수행'], ['때', '모델', '일반', '가능', '검증', '교차', '검증', '수행'], ['교차', '검증', '회', '실시', '평균', '도출', '실험', '결과', '표'], ['모델', '일반', '대출', '신청자', '데이터', '개', '머신', '러닝', '딥', '러닝', '기반', '베이스', '라인', '모델', '성능', '것'], ['값', '통계량', '평가', '지표', '모델', '것', '이', '그래프', '정보', '활용', '신용', '평가', '모형', '채무', '불이행', '예측', '것'], ['금융', '이력', '부족', '씬', '파일러', '데이터', '세트', '베이스', '라인', '모델', '대비', '성능'], ['일부', '베이스', '라인', '모델', '씬', '파일러', '예측', '시', '성능', '데', '모델', '비교', '안정', '성능', '유지'], ['전체', '데이터', '세트', '마찬가지', '씬', '파일러', '데이터', '기준', '모델', '모델', '성능', '것'], ['마지막', '대신', '레이어', '추가', '모델', '대비', '성능', '기록'], ['이', '활용', '범주', '변수', '딩', '채무', '불이행', '예측', '성능', '향상', '도움', '것', '의미'], ['추가', '클래스', '균형', '모델', '영향', '성능', '확인', '소수', '클래스', '다수', '클래스', '비율', '모델', '테스트', '아래', '표', '과', '진행'], ['결과', '클래스', '균형', '상황', '모델', '기존', '모델', '대비', '성능'], ['연구', '모델', '학습', '활용', '개', '핵심', '변수', '중요도', '분석'

In [24]:
okt = Okt()

# Function to extract nouns using Mecab
def extract_okt_nouns(sentences):
    return [okt.nouns(sentence) for sentence in sentences]

# Extract nouns using Mecab
okt_nouns = extract_okt_nouns(quoted_sentences)

# Split texts based on Mecab nouns
texts = [" ".join(nouns).split() for nouns in okt_nouns]

mgp = MovieGroupProcess(K=20, n_iters=200, alpha=0.1, beta=0.1)
V = compute_V(texts)
y = mgp.fit(texts, V)

print(y)

print("----------------------------------- RESULT -----------------------------------")
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 10)

In stage 0: transferred 14 clusters with 7 clusters populated
In stage 1: transferred 7 clusters with 10 clusters populated
In stage 2: transferred 6 clusters with 8 clusters populated
In stage 3: transferred 5 clusters with 9 clusters populated
In stage 4: transferred 5 clusters with 8 clusters populated
In stage 5: transferred 6 clusters with 9 clusters populated
In stage 6: transferred 4 clusters with 10 clusters populated
In stage 7: transferred 7 clusters with 8 clusters populated
In stage 8: transferred 5 clusters with 9 clusters populated
In stage 9: transferred 4 clusters with 8 clusters populated
In stage 10: transferred 6 clusters with 10 clusters populated
In stage 11: transferred 5 clusters with 10 clusters populated
In stage 12: transferred 7 clusters with 8 clusters populated
In stage 13: transferred 5 clusters with 8 clusters populated
In stage 14: transferred 3 clusters with 8 clusters populated
In stage 15: transferred 3 clusters with 8 clusters populated
In stage 16: 

In [36]:
kkma = Kkma()
# Function to extract nouns using Mecab
def extract_kkma_nouns(sentences):
    return [kkma.nouns(sentence) for sentence in sentences]

# Extract nouns using Mecab
kkma_nouns = extract_kkma_nouns(quoted_sentences)

# Split texts based on Mecab nouns
texts = [" ".join(nouns).split() for nouns in kkma_nouns]

mgp = MovieGroupProcess(K=20, n_iters=200, alpha=0.1, beta=0.1)
V = compute_V(texts)
y = mgp.fit(texts, V)

print(y)

print("----------------------------------- RESULT -----------------------------------")
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

# topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-10:][::-1]
print('\nMost important clusters (by number of docs inside):', top_index)
# show the top 5 words in term frequency for each cluster 
top_words(mgp.cluster_word_distribution, top_index, 10)

In stage 0: transferred 14 clusters with 9 clusters populated
In stage 1: transferred 2 clusters with 8 clusters populated
In stage 2: transferred 6 clusters with 8 clusters populated
In stage 3: transferred 5 clusters with 10 clusters populated
In stage 4: transferred 5 clusters with 8 clusters populated
In stage 5: transferred 8 clusters with 8 clusters populated
In stage 6: transferred 7 clusters with 8 clusters populated
In stage 7: transferred 4 clusters with 8 clusters populated
In stage 8: transferred 6 clusters with 9 clusters populated
In stage 9: transferred 4 clusters with 7 clusters populated
In stage 10: transferred 4 clusters with 7 clusters populated
In stage 11: transferred 2 clusters with 8 clusters populated
In stage 12: transferred 1 clusters with 8 clusters populated
In stage 13: transferred 2 clusters with 8 clusters populated
In stage 14: transferred 4 clusters with 10 clusters populated
In stage 15: transferred 7 clusters with 8 clusters populated
In stage 16: tr

HANANUM
In stage 199: transferred 8 clusters with 9 clusters populated
[0, 6, 17, 0, 1, 0, 0, 0, 8, 1, 8, 8, 3, 5, 2, 2, 2, 5, 1, 13]

In stage 199: transferred 5 clusters with 8 clusters populated
[7, 17, 19, 7, 14, 7, 7, 7, 8, 14, 8, 8, 17, 3, 16, 16, 16, 3, 14, 9]

In stage 199: transferred 5 clusters with 8 clusters populated
[10, 1, 1, 10, 2, 10, 10, 10, 10, 2, 8, 8, 6, 15, 12, 12, 12, 18, 2, 15]

In stage 199: transferred 6 clusters with 9 clusters populated
[11, 16, 17, 11, 13, 11, 11, 11, 2, 13, 15, 15, 2, 5, 6, 6, 6, 5, 13, 9]

In stage 199: transferred 5 clusters with 9 clusters populated
[16, 2, 1, 16, 8, 16, 16, 16, 17, 8, 7, 7, 2, 10, 3, 3, 3, 10, 8, 9]

MECAB
In stage 48: transferred 0 clusters with 6 clusters populated
Converged.  Breaking out.
[0, 12, 12, 0, 14, 0, 0, 0, 7, 14, 7, 7, 10, 17, 10, 10, 10, 17, 14, 17]

In stage 45: transferred 0 clusters with 6 clusters populated
Converged.  Breaking out.
[13, 1, 1, 13, 11, 13, 13, 13, 4, 11, 4, 4, 14, 15, 14, 14, 14, 15, 11, 15]

In stage 70: transferred 0 clusters with 7 clusters populated
Converged.  Breaking out.
[16, 19, 19, 16, 15, 16, 16, 16, 10, 15, 10, 10, 8, 0, 9, 9, 9, 8, 15, 0]

In stage 42: transferred 0 clusters with 7 clusters populated
Converged.  Breaking out.
[19, 16, 16, 19, 3, 19, 19, 19, 1, 3, 1, 1, 4, 18, 5, 5, 5, 4, 3, 18]

In stage 38: transferred 0 clusters with 6 clusters populated
Converged.  Breaking out.
[2, 8, 8, 2, 0, 2, 2, 2, 16, 0, 16, 16, 9, 19, 9, 9, 9, 19, 0, 19]

OKT
In stage 29: transferred 0 clusters with 7 clusters populated
Converged.  Breaking out.
[2, 8, 8, 2, 12, 2, 2, 2, 13, 12, 13, 13, 11, 19, 7, 7, 11, 19, 12, 11]

In stage 199: transferred 3 clusters with 9 clusters populated
[1, 3, 9, 1, 12, 1, 1, 1, 18, 12, 18, 18, 11, 19, 11, 11, 11, 19, 13, 17]

In stage 119: transferred 0 clusters with 6 clusters populated
Converged.  Breaking out.
[7, 11, 11, 7, 18, 7, 7, 7, 1, 18, 1, 1, 8, 19, 8, 8, 8, 19, 18, 19]

In stage 199: transferred 7 clusters with 10 clusters populated
[16, 8, 12, 16, 10, 16, 16, 16, 0, 10, 17, 0, 6, 4, 3, 3, 6, 4, 2, 6]

In stage 44: transferred 0 clusters with 6 clusters populated
Converged.  Breaking out.
[19, 3, 3, 19, 18, 19, 19, 19, 9, 18, 9, 9, 12, 4, 12, 12, 12, 4, 18, 4]

KKMA
In stage 199: transferred 5 clusters with 9 clusters populated
[2, 4, 4, 2, 19, 2, 2, 2, 16, 19, 16, 16, 17, 3, 6, 6, 3, 15, 0, 3]

In stage 199: transferred 4 clusters with 9 clusters populated
[10, 9, 9, 10, 2, 10, 10, 10, 0, 2, 16, 0, 3, 5, 19, 19, 5, 13, 2, 5]

In stage 199: transferred 4 clusters with 9 clusters populated
[12, 3, 3, 12, 4, 12, 12, 12, 9, 4, 15, 15, 2, 0, 1, 1, 1, 2, 17, 0]

In stage 199: transferred 4 clusters with 8 clusters populated
[0, 17, 17, 0, 6, 0, 0, 0, 3, 6, 3, 3, 18, 12, 9, 9, 9, 15, 6, 12]

In stage 199: transferred 5 clusters with 9 clusters populated
[10, 6, 6, 10, 15, 10, 10, 10, 16, 15, 16, 16, 9, 13, 5, 5, 5, 0, 11, 13]